In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import os
import torch
from torch.autograd import Variable
from datetime import datetime
import numpy as np

import utils
from vgg19 import Vgg19
import config

import NST



2.3.0


In [2]:
# define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
 # load and reshape the content image
content_img = utils.load_image(os.path.join(config.CONTENT_DIR,config.content_name),config.HEIGHT)
# load and reshape the style image
style_img = utils.load_image(os.path.join(config.STYLE_DIR,config.style_name),config.HEIGHT) 
content_img.shape,style_img.shape

((400, 602, 3), (400, 480, 3))

In [4]:
# use the pretrained model VGG19
model = Vgg19() 

In [5]:
# transform the content image into tensor and more 
content_transformed = NST.transform_image(content_img,device) 
# transform the style image into tensor and more
style_transformed = NST.transform_image(style_img,device) 
content_transformed.shape,style_transformed.shape

(torch.Size([1, 3, 400, 602]), torch.Size([1, 3, 400, 480]))

In [6]:
init_img = content_transformed # define the init image (it can be random noise, content or style)
optimizing_img = Variable(init_img,requires_grad=True) # define the variables who will be optimize to reduce loss (we optimize image's pixels)

In [7]:
content_set_features_map = model(content_transformed)# get the set of features map of content image 
style_set_features_map = model(style_transformed)# get the set of features map of style image 
len(content_set_features_map),len(style_set_features_map)

(6, 6)

In [28]:
print(f"INITIAL: content shape:{content_transformed.shape} | style shape:{style_transformed.shape}")
for i in range(len(content_set_features_map)):
    print(f"{i}:content shape:{content_set_features_map[i].shape} | style shape:{style_set_features_map[i].shape}")

In [29]:
optimizer = torch.optim.LBFGS([optimizing_img],max_iter=config.ITERATION,line_search_fn="strong_wolfe")

In [30]:
target_content = content_set_features_map[model.content_layers]
target_style = [NST.gram_matrix(x) for i,x in enumerate(style_set_features_map) if i in model.style_layers]

In [31]:

def loss_fn(model, target_content, target_style, current_img):
    current_img_set_features_map = model(current_img)

    # Content Loss
    content_loss_fn = torch.nn.MSELoss(reduction="mean")
    content_loss = content_loss_fn(target_content, current_img_set_features_map[4])

    # Style Loss
    style_loss_fn = torch.nn.MSELoss(reduction="sum")
    style_loss = 0.0
    current_style_representation = [NST.gram_matrix(x) for i, x in enumerate(current_img_set_features_map) if i != 4]
    for gram_target, gram_current in zip(target_style, current_style_representation):
        style_loss += style_loss_fn(gram_target, gram_current)
    style_loss /= len(current_style_representation)
    
    #total variation loss
    tv_loss = utils.total_variation(optimizing_img)

    # Total loss
    total_loss = (content_loss * config.CONTENT_WEIGHT) + (style_loss*config.STYLE_WEIGHT) + (tv_loss*config.TV_WEIGHT)
    return total_loss,content_loss,style_loss,tv_loss
    

In [32]:
counter = 0

def optimize_step():
    global counter
    optimizer.zero_grad()
    total_loss,content_loss,style_loss,tv_loss = loss_fn(model, target_content, target_style, optimizing_img)
    total_loss.backward()
    print(f"iteration:{counter} total_loss:{total_loss.item()} content_loss:{content_loss.item()} style_loss:{style_loss.item()} tv_loss:{tv_loss.item()}")
    counter+=1
    return total_loss

# Perform optimization
optimizer.step(optimize_step)
    

In [33]:
final_img = optimizing_img.squeeze().permute(1,2,0).to('cpu').detach().numpy()
final_img = utils.get_uint8_range(final_img)
final_img /=255
final_img.shape

In [34]:
utils.display_image(final_img)

In [35]:
result_name = utils.get_name(config.content_name,config.style_name)
result_img_path = os.path.join(config.RESULT_DIR, f"{result_name}.png")
plt.imsave(result_img_path, final_img)